<a href="https://colab.research.google.com/github/ishwin07/Topsis/blob/main/topsis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import sys
import pandas as pd
import numpy as np
import os

def topsis(input_file, weights, impacts, output_file):

    try:
        if input_file.endswith('.csv'):
            data = pd.read_csv(input_file)
        elif input_file.endswith('.xlsx'):
            data = pd.read_excel(input_file)
        else:
            print("Error: Input file must be .csv or .xlsx")
            sys.exit(1)
    except FileNotFoundError:
        print("Error: Input file not found")
        sys.exit(1)


    if data.shape[1] < 3:
        print("Error: Input file must contain three or more columns")
        sys.exit(1)

    decision_matrix = data.iloc[:, 1:]


    try:
        decision_matrix = decision_matrix.astype(float)
    except:
        print("Error: From 2nd to last columns must contain numeric values only")
        sys.exit(1)

    weights = weights.split(',')
    impacts = impacts.split(',')

    if len(weights) != decision_matrix.shape[1] or len(impacts) != decision_matrix.shape[1]:
        print("Error: Number of weights, impacts and columns must be same")
        sys.exit(1)

    try:
        weights = np.array(weights, dtype=float)
    except:
        print("Error: Weights must be numeric")
        sys.exit(1)

    for impact in impacts:
        if impact not in ['+', '-']:
            print("Error: Impacts must be either '+' or '-'")
            sys.exit(1)


    norm_matrix = decision_matrix / np.sqrt((decision_matrix ** 2).sum())


    weighted_matrix = norm_matrix * weights


    ideal_best = []
    ideal_worst = []

    for i in range(len(impacts)):
        if impacts[i] == '+':
            ideal_best.append(weighted_matrix.iloc[:, i].max())
            ideal_worst.append(weighted_matrix.iloc[:, i].min())
        else:
            ideal_best.append(weighted_matrix.iloc[:, i].min())
            ideal_worst.append(weighted_matrix.iloc[:, i].max())

    ideal_best = np.array(ideal_best)
    ideal_worst = np.array(ideal_worst)


    dist_best = np.sqrt(((weighted_matrix - ideal_best) ** 2).sum(axis=1))
    dist_worst = np.sqrt(((weighted_matrix - ideal_worst) ** 2).sum(axis=1))


    topsis_score = dist_worst / (dist_best + dist_worst)


    data['Topsis Score'] = topsis_score
    data['Rank'] = data['Topsis Score'].rank(ascending=False).astype(int)


    if output_file.endswith('.csv'):
        data.to_csv(output_file, index=False)
    elif output_file.endswith('.xlsx'):
        data.to_excel(output_file, index=False)
    else:
        print("Error: Output file must be .csv or .xlsx")
        sys.exit(1)

    print("TOPSIS analysis completed successfully")



